### 🟢 Less messy MVP that works (most functions were split into modules)

🔴 Note: lots of profiling/logging dependencies and debug messages to trace all matrices and compare with nexmd

In [1]:
%load_ext autoreload

%autoreload 2
%reload_ext autoreload

# === IMPORTS ===
sys.path.append('/vast/projects/ml4chem/NikitaFedik/GITHUB/PYSEQM_davidson')
import logging, sys
import torch
import seqm
from ase.io import read as ase_read
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.ElectronicStructure import Electronic_Structure
from termcolor import colored


from seqm.seqm_functions.fock import fock
from seqm.seqm_functions.pack import unpack
import seqm.seqm_functions.pack as pack
import torch.nn.functional as F

#=== TORCH OPTIONS ===

device = torch.device('cpu')

torch.set_default_dtype(torch.float64)
# if torch.cuda.is_available():
#     device = torch.device('cuda')
# else:
#     device = torch.device('cpu')
dtype = torch.float64
# torch.set_printoptions(precision=5, linewidth=200, profile="full", sci_mode=False)
torch.set_printoptions(precision=5, linewidth=200, sci_mode=False, profile = 'short')

NameError: name 'sys' is not defined

### NOTES

1) QM part seems to be working fine
2) full diagonalization agrees with NEXMD
3) Davidson eigenvalues almost always agree with nexmd and ORCA (depending on starting and max subspace size, number of eigenvalues to keep and when to collapse subspace, some degenrate eigenvalues are sometimes missed) - basically, small guess space misses relevant vectors, but large guess includes them (probably typical for Krylo algorithms)
4) be careful with G_antisym routine - took ideas from PASCAL 1 (nexmd) - might want double check all martrix values for different test cases

In [98]:
device

device(type='cpu')

In [99]:
# colored logging with custom level QM for deeper routines

logging.basicConfig(level=logging.DEBUG,
                    format='%(funcName)s : %(lineno)d : %(levelname)s : %(message)s')

QM1 = evel=logging.DEBUG - 3 # informal level of depth; QM1 - almost always, usually outside of loops
QM2 = evel=logging.DEBUG - 4 #                          QM2 - sometimes, in the loops
QM3 = evel=logging.DEBUG - 5

logging.addLevelName(QM1, "QM1")
def qm1(self, message, *args, **kwargs):
    if self.isEnabledFor(QM1 ):
        self._log(QM1, message, args, **kwargs) 
        
logging.addLevelName(QM2, "QM2")
def qm2(self, message, *args, **kwargs):
    if self.isEnabledFor(QM2):
        self._log(QM2, message, args, **kwargs) 
 
logging.addLevelName(QM3, "QM3")
def qm3(self, message, *args, **kwargs):
    if self.isEnabledFor(QM3 ):
        self._log(QM3, message, args, **kwargs) 
           
        
logging.Logger.qm1 = qm1   
logging.Logger.qm2 = qm2
logging.Logger.qm3 = qm3
  
logger = logging.getLogger()

                              
colors = {'qm'        : ('cyan',     None, None),
          'matrix'    : ('blue',     None, ['bold']),
          'vector'    : ('yellow',   None, ['bold']),
          'evals'     : ('green',    None, ['bold']),
          'warn'     : ('red',    None, ['bold'])
          }

def fmt_log(data, message, fmt):
    """
    fmt_log : formats log message with color and style using termcolor module

    Args:
        data (any): data to print
        message (str or None): message to print, pass None if no message is needed
        fmt (str): style from colors dict

    Returns:
        str: formatted string with color and style
    """    

    if type(data) is list or type(data) is tuple or type(data) is torch.Tensor:
        
        mes = f'{colored(message, colors[fmt][0], colors[fmt][1], attrs=colors[fmt][2])}\n' # add new line to align array
    else:
        mes = f'{colored(message, colors[fmt][0], colors[fmt][1], attrs=colors[fmt][2])} : '
        
    if data == None:
        return mes
    else:
        return mes + str(colored(data, colors[fmt][0], colors[fmt][1], attrs=colors[fmt][2]))

### QM routines

In [2]:
def run_seqm_1mol(xyz):
    """
    run_seqm_1mol : run PYSEQM for a single molecule

    Args:
        xyz (str): path to xyz file

    Returns:
        Molecule object: PYSEQM object with molecule data
    """    
    
    atoms = ase_read(xyz)
    species = torch.tensor([atoms.get_atomic_numbers()], dtype=torch.long, device=device)
    coordinates = torch.tensor([atoms.get_positions()], dtype=dtype, device=device)
    
    const = Constants().to(device)

    elements = [0]+sorted(set(species.reshape(-1).tolist()))

    seqm_parameters = {
                    'method' : 'PM3',  # AM1, MNDO, PM#
                    'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                    'scf_converger' : [2,0.0], # converger used for scf loop
                                            # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                            # [1], adaptive mixing
                                            # [2], adaptive mixing, then pulay
                    'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                                #[True, eps] or [False], eps for SP2 conve criteria
                    'elements' : elements, #[0,1,6,8],
                    'learned' : [], # learned parameters name list, e.g ['U_ss']
                    #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                    'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                    'eig' : True,
                    'excited' : True,
                    }

    mol = seqm.Molecule.Molecule(const, seqm_parameters, coordinates, species).to(device)

    ### Create electronic structure driver:
    esdriver = Electronic_Structure(seqm_parameters).to(device)

    ### Run esdriver on m:
    esdriver(mol)
    
    return mol

In [101]:
def run_seqm(xyz_list):
    """
    run_seqm_1mol : run PYSEQM for a single molecule

    Args:
        xyz (str): path to xyz file

    Returns:
        Molecule object: PYSEQM object with molecule data
    """    
    
    atoms = [ase_read(x) for x in xyz_list]
    species = torch.tensor([atoms[x].get_atomic_numbers() for x in range(len(atoms))], dtype=torch.long, device=device)
    coordinates = torch.tensor([atoms[x].get_positions() for x in range(len(atoms))], dtype=dtype, device=device)
    
    const = Constants().to(device)

    elements = [0]+sorted(set(species.reshape(-1).tolist()))

    seqm_parameters = {
                    'method' : 'PM3',  # AM1, MNDO, PM#
                    'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                    'scf_converger' : [2,0.0], # converger used for scf loop
                                            # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                            # [1], adaptive mixing
                                            # [2], adaptive mixing, then pulay
                    'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                                #[True, eps] or [False], eps for SP2 conve criteria
                    'elements' : elements, #[0,1,6,8],
                    'learned' : [], # learned parameters name list, e.g ['U_ss']
                    #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                    'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                    'eig' : True,
                    'excited' : True,
                    }

    mol = seqm.Molecule.Molecule(const, seqm_parameters, coordinates, species).to(device)

    ### Create electronic structure driver:
    esdriver = Electronic_Structure(seqm_parameters).to(device)

    ### Run esdriver on m:
    esdriver(mol)
    
    return mol

### DAVIDSON routines

In [104]:
logger.setLevel(logging.DEBUG)  # custom logging level; lower than DEBUG
                               # printed above QM (QM, DEBUG, INFO, etc)

In [170]:
# ALL new modules translated from NEXMD
# keep as seqm functions 

from seqm.seqm_functions.excited import ortho as orthogonalize
from seqm.seqm_functions.excited.hamiltonian import gen_V
from seqm.seqm_functions.excited.hamiltonian import form_cis
import seqm.seqm_functions.excited
from seqm.seqm_functions.excited.orb_transform import mo2ao
from seqm.seqm_functions.excited.ortho import orthogonalize_matrix

In [176]:
def davidson(device, mol, N_exc, keep_n, n_V_max,  max_iter, tol):
    """
    Davidson algorithm for solving eigenvalue problem of large sparse diagonally dominant matrices
    Hamiltonian is not generated or stored explicitly, only matrix-vector products are used on-the fly:
    guess space V should be orthogonalized at each iteration
    M (projection of smaller size) is V.T @ H @ V 
    #! RPA (TDHF) is not implemented yet, non-Hermitian (non-symmetric), requires also left eigenvectors 
    note that notation differes between implementations: V.T x A x V is bAb
    # TODO: 1) check if convergence of e_vals is needed
    # TODO: 2) vectorize and optimize orthogonalization
    # TODO: 3) check if some vectors should be dropped 
    # TODO: 4) eliminate loops 
    # TODO: 5) check if whole M should be regenerated, or only sub-blocks corresponding to new guess vectors
    # TODO: 6) add parameter checker like Krylov dims << N_cis

    Args:
        mol (PYSEQM object): object to hold all qm data from PYSEQM
        N_exc (int)        : number of excited states to calculate
        keep_n (int)       : number of e_vals, e_vecs to keep at each iteration
        n_V_max (int)      : maximum size of Krylov subspace, 
                             projected matrix will be no more than M(n_V_max x n_V_max)
        max_iter (int)     : maximum number of iterations in Davidson
        tol (float)        : treshold for residual
        
    Returns:
        tuple of tensors: eigenvalues (excitation energies in default units, eV) and eigenvectors 
    """    
    
    n_V_start = N_exc * 2 # dimension of Krylov subspace, analogue of nd1  
    N_cis = mol.nocc * mol.nvirt
    N_rpa = 2 * N_cis
    
    term = False  # terminate algorithm
    iter = 0
    L_xi = torch.zeros((N_cis, n_V_start), device=device)
    V = gen_V(device, mol, N_cis, n_V_start) # generate initial guess, V here #! should be renamed
    diag = None # create diagonal of M only once
    
    while iter < max_iter and not term: # Davidson loop
        
        if iter > 0: # skip first step, as initial V is orthogonal
            V = orthogonalize_matrix(V)
            
        print('=================================', flush=True)
        print(colored(f' ITERATION : {iter} ', 'red', 'on_white', attrs=['bold']), flush=True)
        print('SUBSPACE SIZE V: ', V.shape, flush=True)
        print('=================================')
       
        # ---------- form A x b product --------------------
        L_xi = torch.zeros((N_cis, V.shape[1]), device=device) #! NOT iter here

        logger.qm1(fmt_log(V, 'V BEFORE L_xi after ORTO', 'qm'))
        for i in range(V.shape[1]): 
            logger.qm3('Lxi iterations=%s', i)
            # L_xi[:,i] = form_L_xi(V[:,i], mol, N_cis, N_rpa)
            L_xi[:,i] = form_cis(device, V[:,i], mol, N_cis, N_rpa) # TODO: should be vectorized  

    
        right_V = L_xi #[N_cis:] # (A)b 
        
        # logger.qm1(fmt_log(right_V.shape, 'right_V shape', 'matrix'))
        # logger.qm1(fmt_log(right_V, 'right_V', 'matrix'))       
        # ---------- form b.T x Ab product --------------------
        
        M =  V.T @ right_V
        
        # logger.debug(fmt_log(M.shape, 'M shape', 'qm'))
        # logger.debug(fmt_log(M, 'M', 'qm'))
        if iter == 0:
            diag = torch.diag(M) # create diagonal only once
            
        iter += 1
        
        logger.qm1(fmt_log(diag, 'diag', 'qm'))
    
        # ---------- diagonalize projection M --------------------
        r_eval, r_evec = torch.linalg.eigh(M) # find eigenvalues and eigenvectors
       
        r_eval = r_eval.real
        r_evec = r_evec.real
        r_eval, r_idx = torch.sort(r_eval, descending=False) # sort eigenvalues in ascending order
        logger.debug(fmt_log(r_eval, 'RIGHT EVALS', 'evals'))
        r_evec = r_evec[:, r_idx] # sort eigenvectors accordingly
    
        e_val_n = r_eval[:keep_n] # keep only the lowest keep_n eigenvalues; full are still stored as e_val
        e_vec_n = r_evec[:, :keep_n]
        resids = torch.zeros(V.shape[0], len(e_val_n)) # account for left and right evecs

        # ---------- calculate residual vectors --------------------
        for j in range(len(e_val_n)): # calc residuals 
            resids[:,j] = right_V @ e_vec_n[:,j] - e_val_n[j] * (V @ e_vec_n[:,j])
            
       # logger.debug(fmt_log(resids, 'resids', 'matrix'))     
        resids_norms_r = torch.tensor([resids[:,x].norm() for x in range(resids.shape[1])])

        # ---------- expand guess space V buy not-converged resids --------------------
        # !!! PROBABLY HIGHLY INEFFICIENT TODO: vectorize !!! 
        if torch.any(resids_norms_r > tol):
            mask_r = resids_norms_r >= tol
            large_res_r = resids[:,mask_r] # residuals larger than tol
           # logger.debug(fmt_log(large_res_r, 'LARGE RESIDUALS', 'vector'))           
            large_res_r.to(device)
            cor_e_val_r = e_val_n[mask_r] # corresponding eigenvalues !!! check if matches
            
            # ------keep adding new resids --------------------
            if V.shape[1] <= n_V_max:     

                    for j in range(large_res_r.shape[1]):
                        if V.shape[1] <= n_V_max:
                            s = large_res_r[:,j] # conditioned residuals > tol

                            if s.norm() >= tol:
                                logger.debug(fmt_log((s.norm().item()), 'NORM of RESIDUAL', 'warn'))
                                denom = (diag[j] - cor_e_val_r[j])
                                denom.to(device) 
                                s = s/denom # conditioned residuals
                                s.to(device)
                                # logger.debug(fmt_log(s.norm(), 'NORM OF NEW RESIDUAAL', 'vector'))
                                V = torch.column_stack((V, s/s.norm()))
                            else:
                                pass
            # ------ collapse (restart) if space V is too large; mix eigenvectors with V------------
            else:
                logger.debug(fmt_log(None, '!!!! MAX subspace reached !!!!', 'warn'))
                #logger.debug(fmt_log(V, 'V before collapse', 'qm'))

                V =  V @ r_evec[:, :n_V_start]
                logger.debug(fmt_log(V.shape, 'V shape after restart', 'qm'))
                #logger.debug(fmt_log(V, 'V AFTER collapse', 'qm'))

                continue

        else:
            term = True
            print('============================', flush=True)
            print('all residuals are below tolerance')
            print('DAVIDSON ALGORITHM CONVERGED', flush=True)
            print('============================', flush=True)

            return r_eval, r_evec

    # runs after big loop if did not converge
    print('============================', flush=True)
    print('!!! DAVIDSON ALGORITHM DID NOT CONVERGE !!!', flush=True)
    print('============================', flush=True)
    
    return r_eval, r_evec

### **INTERACTIVE TESTING**

In [186]:
%reload_ext autoreload
%autoreload 2


logger.debug(fmt_log(eval, 'FINAL eval ', 'evals'))
from seqm.seqm_functions.excited import ortho as orthogonalize
from seqm.seqm_functions.excited.hamiltonian import gen_V
from seqm.seqm_functions.excited.hamiltonian import form_cis
from seqm.seqm_functions.excited.orb_transform import mo2ao



mol = run_seqm_1mol('/vast/projects/ml4chem/NikitaFedik/GITHUB/PYSEQM_davidson/c6h6.xyz')
eval, _ = davidson(device = 'cpu', 
                   mol = mol, 
                   N_exc = 3,
                   keep_n = 4,
                   n_V_max = 40, 
                   max_iter = 200, 
                   tol = 1e-6)

logger.debug(fmt_log(eval, 'FINAL eval ', 'evals'))

<module> : 13 : DEBUG : FINAL eval 
tensor([ 4.01924,  4.24138,  5.93420,  5.96212,  5.96275,  6.28559,  6.31778,  6.71974,  7.16728,  8.30548,  9.27592, 10.94060, 13.13104, 15.23407, 17.27802, 18.62388, 20.11723, 24.29461, 25.83632,
        28.49596, 29.30479, 32.24481, 34.87008, 36.37702, 37.36901, 38.35180, 38.39349, 38.80589], grad_fn=<SortBackward0>)


 ITERATION : 0 
SUBSPACE SIZE V:  torch.Size([225, 6])


davidson : 101 : DEBUG : RIGHT EVALS
tensor([5.12439, 5.34620, 6.98829, 7.01342, 7.01436, 7.37410], grad_fn=<SortBackward0>)
davidson : 132 : DEBUG : NORM of RESIDUAL : 1.4043994385202523
davidson : 132 : DEBUG : NORM of RESIDUAL : 1.45565761707566
davidson : 132 : DEBUG : NORM of RESIDUAL : 1.3550453720699032
davidson : 132 : DEBUG : NORM of RESIDUAL : 0.6809422235258806


 ITERATION : 1 
SUBSPACE SIZE V:  torch.Size([225, 10])


davidson : 101 : DEBUG : RIGHT EVALS
tensor([ 4.67871,  4.90330,  6.53368,  6.94334,  7.01436,  7.37410,  9.54983, 10.13041, 11.02730, 13.63008], grad_fn=<SortBackward0>)
davidson : 132 : DEBUG : NORM of RESIDUAL : 1.5959327467640572
davidson : 132 : DEBUG : NORM of RESIDUAL : 1.5256143152816022
davidson : 132 : DEBUG : NORM of RESIDUAL : 1.353134890200715
davidson : 132 : DEBUG : NORM of RESIDUAL : 0.7503089513603357


...

davidson : 101 : DEBUG : RIGHT EVALS
tensor([ 4.01924,  4.24138,  5.93420,  5.96212,  5.96275,  6.28559,  6.32680,  6.77063,  7.28625,  8.35393,  9.61237, 11.82448, 14.15355, 16.72470, 18.29610, 18.72838, 22.86496, 25.16552, 26.57824,
        28.85993, 30.08327, 33.84144, 36.32253, 37.05537, 38.22192, 38.39349, 38.79619], grad_fn=<SortBackward0>)
davidson : 132 : DEBUG : NORM of RESIDUAL : 1.1127352643852443e-06


 ITERATION : 66 
SUBSPACE SIZE V:  torch.Size([225, 28])


davidson : 101 : DEBUG : RIGHT EVALS
tensor([ 4.01924,  4.24138,  5.93420,  5.96212,  5.96275,  6.28559,  6.31778,  6.71974,  7.16728,  8.30548,  9.27592, 10.94060, 13.13104, 15.23407, 17.27802, 18.62388, 20.11723, 24.29461, 25.83632,
        28.49596, 29.30479, 32.24481, 34.87008, 36.37702, 37.36901, 38.35180, 38.39349, 38.80589], grad_fn=<SortBackward0>)


all residuals are below tolerance
DAVIDSON ALGORITHM CONVERGED


<module> : 30 : DEBUG : FINAL eval 
tensor([ 4.01924,  4.24138,  5.93420,  5.96212,  5.96275,  6.28559,  6.31778,  6.71974,  7.16728,  8.30548,  9.27592, 10.94060, 13.13104, 15.23407, 17.27802, 18.62388, 20.11723, 24.29461, 25.83632,
        28.49596, 29.30479, 32.24481, 34.87008, 36.37702, 37.36901, 38.35180, 38.39349, 38.80589], grad_fn=<SortBackward0>)
